<a href="https://colab.research.google.com/github/jhj1819/study_ai/blob/main/Getting_Started_with_Titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'titanic:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-competitions-data%2Fkaggle-v2%2F3136%2F26502%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240926%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240926T062951Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D5d1e5e3048d2e55b5cfaf57e5c27392090af417a0ce32f13f5caef7709b461db07f68f28db5e6b26564ca56cb64f26f6e2dbc8c4f016b797ee8711a6d2afc77537f0fd7eb63d0a7a31a72f30148c3a3c65a7d8c420d8ce9146c2ef7ff100972db777c0148a96a501c49e0e552f9a94f45e76f2c9babdb85f02c49171408aa653ea2e169053fcfe2a52a83cccc1fb0e8632cd1ab7f03581ed0b221e8f31f482729b60e939b3bd966a0ff13629a9a59a79d4744243f7e6b6194fffefba179cd852184cd5f09775af28dfb991839606bdd83fa14791c1ea1180495b3e8f3de27b8633113b118a30d2545e661790fd2c6e2c4eaee2e805d26cb800456da9cc5a9fff'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 34877 bytes downloaded
Downloaded and uncompressed: titanic
Data source import complete.


In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [3]:
train_data = pd.read_csv("/kaggle/input/titanic/train.csv") # 훈련데이터 가져오기
train_data.head() # 5개 데이터만 반환

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
test_data = pd.read_csv("/kaggle/input/titanic/test.csv") # 테스트데이터 가져오기
test_data.head()


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [5]:
type(test_data) # test_data의 타입은 DataFrame

pandas.core.frame.DataFrame

### loc 메서드 :
첫번째에 행 정보, 두번째에 열 정보

### women의 구조
: [승객번호가 아니라 인덱스임][Survived(0 or 1)]

In [6]:
women = train_data.loc[train_data.Sex == 'female']["Survived"] # 성별이 female이고, Survived컬럼에 대해서..?
# women의 구조: 2차원구조 [train_data에서의 인덱스][Survived컬럼(0 or 1)]
rate_women = sum(women)/len(women) # 살아있는 여자 수/ 전체 여자 수

print("% of women who survived:", rate_women)

% of women who survived: 0.7420382165605095


women의 형태

In [7]:
women.head()

,Survived
1,1
2,1
3,1
8,1
9,1


### Series
컬럼이 한개인 DataFrame

In [8]:
type(women) # 딕셔너리구조?

pandas.core.series.Series

In [9]:
men = train_data.loc[train_data.Sex == 'male']["Survived"]
rate_men = sum(men)/len(men)

print("% of men who survived:", rate_men)

% of men who survived: 0.18890814558058924


## 머신러닝의 전처리 작업
### 1. 수치형 데이터로 변환
ex) 월:1, 화:2, 수:3
### 2. 수치화된 데이터를 가변수화

### *왜 수치화 뿐만 아니라 가변수화까지 하는가?
왜냐하면 수치형 데이터로만 변환을 할 시 서로 간의 관계성이 생기게 된다.
예를들어 월, 화, 수를 1,2,3 으로 나타내면 , 1+2 = 3 이라는 관계성이 존재하게된다.
하지만, 실제 데이터인 월,화,수에는 그러한 관계성이 없다.
즉, 사실이 아닌 관계성으로 잘못된 학습이 일어날 수 있으므로 서로 무관한 수, 즉 더미로 만든 가변수로 변환함으로서 그러한 문제를 막아줄 수 있다.

pandas에서는 쉽게 더미의 가변수를 만들 수 있도록 get_dummies함수를 제공한다.
![image.png](attachment:11bf043f-6c68-4e92-b53b-54a5ff43eef8.png)

![image.png](attachment:68de02e3-a6ca-4b4b-850a-338b0ee98af7.png)

In [10]:
from sklearn.ensemble import RandomForestClassifier

y = train_data["Survived"]
y


,Survived
0,0
1,1
2,1
3,1
4,0
...,...
886,0
887,1
888,0
889,1


In [11]:
features = ["Pclass", "Sex", "SibSp", "Parch"]
X = pd.get_dummies(train_data[features])
X


,Pclass,SibSp,Parch,Sex_female,Sex_male
0,3,1,0,False,True
1,1,1,0,True,False
2,3,0,0,True,False
3,1,1,0,True,False
4,3,0,0,False,True
...,...,...,...,...,...
886,2,0,0,False,True
887,1,0,0,True,False
888,3,1,2,True,False
889,1,0,0,False,True


In [12]:
X_test = pd.get_dummies(test_data[features])
X_test


,Pclass,SibSp,Parch,Sex_female,Sex_male
0,3,0,0,False,True
1,3,1,0,True,False
2,2,0,0,False,True
3,3,0,0,False,True
4,3,1,1,True,False
...,...,...,...,...,...
413,3,0,0,False,True
414,1,0,0,True,False
415,3,0,0,False,True
416,3,0,0,False,True


In [13]:
model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=1) # 결정트리 100개, 깊이는 5로 제한.
model.fit(X,y)
predictions = model.predict(X_test)

output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': predictions})
output.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!
